In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

DIM = (176,176)

work_dr = IDG()

train_ds = train_datagen.flow_from_directory(
        "/kaggle/input/knee-oa-merged/Dataset_merged/train",
        seed=123,
        target_size=(176, 176),
        batch_size=64,
        class_mode='sparse'
        )

test_ds = test_datagen.flow_from_directory(
"/kaggle/input/knee-oa-merged/Dataset_merged/test",
seed=123,
target_size=(176, 176),
batch_size=64,
    class_mode='sparse'
)

val_ds = val_datagen.flow_from_directory(
"/kaggle/input/knee-oa-merged/Dataset_merged/val",
seed=123,
target_size=(176, 176),
batch_size=64,
class_mode='sparse'
)

Found 5778 images belonging to 3 classes.
Found 1656 images belonging to 3 classes.
Found 826 images belonging to 3 classes.


In [4]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=10, 
                               restore_best_weights=True)

In [5]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_schedule = LearningRateScheduler(lr_scheduler)

## DenseNet169

*Thomas, Kevin A., et al. "Automated classification of radiographic knee osteoarthritis severity using deep neural networks." Radiology: Artificial Intelligence 2.2 (2020): e19006*

In [26]:
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

denseNet169 = DenseNet169(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
denseNet169.trainable = True
denseNet169_model_new = models.Sequential()
denseNet169_model_new.add(denseNet169)
denseNet169_model_new.add(layers.GlobalAveragePooling2D())
denseNet169_model_new.add(layers.Dense(128, activation='relu',  kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
denseNet169_model_new.add(layers.Dropout(0.3))
denseNet169_model_new.add(layers.Dense(5, activation='softmax'))  

51877672/51877672 [==============================] - 0s 0us/step


In [27]:
denseNet169_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [28]:
denseNet169_new = denseNet169_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 191s 1s/step - loss: 2.1579 - accuracy: 0.5107 - val_loss: 79.6365 - val_accuracy: 0.3947 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 47s 515ms/step - loss: 1.1940 - accuracy: 0.5964 - val_loss: 1.4230 - val_accuracy: 0.4310 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 47s 519ms/step - loss: 0.9773 - accuracy: 0.6011 - val_loss: 0.9340 - val_accuracy: 0.5847 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 46s 507ms/step - loss: 0.8400 - accuracy: 0.6317 - val_loss: 1.2171 - val_accuracy: 0.5194 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 47s 515ms/step - loss: 0.7800 - accuracy: 0.6449 - val_loss: 1.0476 - val_accuracy: 0.5327 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 47s 511ms/step - loss: 0.7293 - accuracy: 0.6582 - val_loss: 0.8352 - val_accuracy: 0.6344 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 46s 506ms/ste

In [29]:
loss, accuracy = denseNet169_model_new.evaluate(test_ds)

26/26 [==============================] - 16s 639ms/step - loss: 0.5202 - accuracy: 0.7566


## VGG16

*Alshamrani, Hassan A., et al. "Osteo-NeT: An Automated System for Predicting Knee Osteoarthritis from X-ray Images Using Transfer-Learning-Based Neural Networks Approach." Healthcare. Vol. 11. No. 9. MDPI, 2023.*

In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
vgg16.trainable = True
vgg16_model_new = models.Sequential()
vgg16_model_new.add(vgg16)
vgg16_model_new.add(layers.GlobalAveragePooling2D())
vgg16_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
vgg16_model_new.add(layers.Dropout(0.3))
vgg16_model_new.add(layers.Dense(5, activation='softmax'))  

58889256/58889256 [==============================] - 0s 0us/step


In [10]:
vgg16_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [11]:
vgg16_new = vgg16_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 74s 689ms/step - loss: 255.5651 - accuracy: 0.4038 - val_loss: 1.5885 - val_accuracy: 0.3971 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 62s 682ms/step - loss: 2562.3933 - accuracy: 0.4154 - val_loss: 1.3840 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 62s 684ms/step - loss: 14148.3701 - accuracy: 0.3993 - val_loss: 1.3382 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 63s 686ms/step - loss: 3.6902 - accuracy: 0.3832 - val_loss: 1.3271 - val_accuracy: 0.3971 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 63s 686ms/step - loss: 3.4078 - accuracy: 0.3679 - val_loss: 1.7539 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 62s 678ms/step - loss: 1.8106 - accuracy: 0.4417 - val_loss: 1.6372 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 62s

In [13]:
loss, accuracy = vgg16_model_new.evaluate(test_ds)

26/26 [==============================] - 3s 120ms/step - loss: 1.3350 - accuracy: 0.3859


## ResNet101

*Mohammed, Abdul Sami, et al. "Knee Osteoarthritis Detection and Severity Classification Using Residual Neural Networks on Preprocessed X-ray Images." Diagnostics 13.8 (2023): 1380.*

In [35]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

resnet101 = ResNet101(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet101.trainable = True
resnet101_model_new = models.Sequential()
resnet101_model_new.add(resnet101)
resnet101_model_new.add(layers.GlobalAveragePooling2D())
resnet101_model_new.add(layers.Dense(128, activation='relu',kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
resnet101_model_new.add(layers.Dropout(0.3))
resnet101_model_new.add(layers.Dense(5, activation='softmax'))  

171446536/171446536 [==============================] - 1s 0us/step


In [36]:
resnet101_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [37]:
resnet101_new = resnet101_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 123s 591ms/step - loss: 2.5448 - accuracy: 0.4708 - val_loss: 1.6997 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 47s 516ms/step - loss: 1.2295 - accuracy: 0.5540 - val_loss: 1.6748 - val_accuracy: 0.3910 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 49s 531ms/step - loss: 0.9070 - accuracy: 0.6019 - val_loss: 1.4004 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 48s 524ms/step - loss: 0.7917 - accuracy: 0.6265 - val_loss: 1.3251 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 48s 519ms/step - loss: 0.7797 - accuracy: 0.6327 - val_loss: 1.2641 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 48s 526ms/step - loss: 0.7396 - accuracy: 0.6426 - val_loss: 1.1223 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 48s 524ms/s

In [38]:
loss, accuracy = resnet101_model_new.evaluate(test_ds)

26/26 [==============================] - 5s 191ms/step - loss: 0.5591 - accuracy: 0.7391


## VGG19

*Chen, Pingjun, et al. "Fully automatic knee osteoarthritis severity grading using deep neural networks with a novel ordinal loss." Computerized Medical Imaging and Graphics 75 (2019): 84-92.*

In [39]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
vgg19.trainable = True
vgg19_model_new = models.Sequential()
vgg19_model_new.add(vgg19)
vgg19_model_new.add(layers.GlobalAveragePooling2D())
vgg19_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
vgg19_model_new.add(layers.Dropout(0.3))
vgg19_model_new.add(layers.Dense(5, activation='softmax'))  

80134624/80134624 [==============================] - 0s 0us/step


In [40]:
vgg19_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [41]:
vgg19_new = vgg19_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 51s 501ms/step - loss: 246.9236 - accuracy: 0.4171 - val_loss: 1.3380 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 46s 504ms/step - loss: 10485.0918 - accuracy: 0.4093 - val_loss: 1.2166 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 47s 509ms/step - loss: 14.7478 - accuracy: 0.4244 - val_loss: 1.2016 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 46s 497ms/step - loss: 75.5310 - accuracy: 0.4330 - val_loss: 1.1612 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 46s 499ms/step - loss: 1.1680 - accuracy: 0.4275 - val_loss: 1.1487 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 46s 504ms/step - loss: 1.1565 - accuracy: 0.4306 - val_loss: 1.1412 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 46s 

In [42]:
loss, accuracy = vgg19_model_new.evaluate(test_ds)

26/26 [==============================] - 4s 162ms/step - loss: 1.1022 - accuracy: 0.4487


# Proposed Models

## ResNet152

In [14]:
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

resnet152 = ResNet152(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet152.trainable = True
resnet152_model_new = models.Sequential()
resnet152_model_new.add(resnet152)
resnet152_model_new.add(layers.GlobalAveragePooling2D())
resnet152_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
resnet152_model_new.add(layers.Dropout(0.3))
resnet152_model_new.add(layers.Dense(5, activation='softmax'))  

In [15]:
resnet152_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [16]:
resnet152_new = resnet152_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 184s 758ms/step - loss: 2.6688 - accuracy: 0.4685 - val_loss: 4.8900 - val_accuracy: 0.3971 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 67s 727ms/step - loss: 1.3184 - accuracy: 0.5744 - val_loss: 1.2873 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 67s 725ms/step - loss: 0.9445 - accuracy: 0.5950 - val_loss: 1.1238 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 66s 725ms/step - loss: 0.8201 - accuracy: 0.6282 - val_loss: 1.0852 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 66s 722ms/step - loss: 0.7856 - accuracy: 0.6333 - val_loss: 1.0991 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 65s 714ms/step - loss: 0.7544 - accuracy: 0.6296 - val_loss: 1.0862 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 66s 718ms/s

In [17]:
loss, accuracy = resnet152_model_new.evaluate(test_ds)

26/26 [==============================] - 5s 200ms/step - loss: 0.5507 - accuracy: 0.7446


## ResNet50

In [18]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet50.trainable = True
resnet50_model_new = models.Sequential()
resnet50_model_new.add(resnet50)
resnet50_model_new.add(layers.GlobalAveragePooling2D())
resnet50_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
resnet50_model_new.add(layers.Dropout(0.3))
resnet50_model_new.add(layers.Dense(5, activation='softmax'))  

94765736/94765736 [==============================] - 0s 0us/step


In [19]:
resnet50_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [20]:
resnet50_new = resnet50_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 98s 666ms/step - loss: 2.5963 - accuracy: 0.5332 - val_loss: 1.9688 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 60s 657ms/step - loss: 1.2546 - accuracy: 0.5990 - val_loss: 1.3959 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 60s 654ms/step - loss: 0.9044 - accuracy: 0.6258 - val_loss: 1.1489 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 62s 675ms/step - loss: 0.7970 - accuracy: 0.6461 - val_loss: 1.1375 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 61s 668ms/step - loss: 0.7320 - accuracy: 0.6487 - val_loss: 1.0732 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 61s 665ms/step - loss: 0.7285 - accuracy: 0.6577 - val_loss: 1.0660 - val_accuracy: 0.4419 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 61s 666ms/st

In [21]:
loss, accuracy = resnet50_model_new.evaluate(test_ds)

26/26 [==============================] - 5s 201ms/step - loss: 0.5339 - accuracy: 0.7572


## DenseNet121

In [6]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

denseNet121 = DenseNet121(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
denseNet121.trainable = True
denseNet121_model_new = models.Sequential()
denseNet121_model_new.add(denseNet121)
denseNet121_model_new.add(layers.GlobalAveragePooling2D())
denseNet121_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
denseNet121_model_new.add(layers.Dropout(0.3))
denseNet121_model_new.add(layers.Dense(5, activation='softmax'))  

29084464/29084464 [==============================] - 1s 0us/step


In [7]:
denseNet121_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [8]:
denseNet121_new = denseNet121_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 137s 773ms/step - loss: 2.3874 - accuracy: 0.4898 - val_loss: 1.8797 - val_accuracy: 0.4504 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 55s 598ms/step - loss: 1.3382 - accuracy: 0.6009 - val_loss: 1.5185 - val_accuracy: 0.4831 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 55s 601ms/step - loss: 1.0289 - accuracy: 0.6260 - val_loss: 1.3696 - val_accuracy: 0.5097 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 56s 608ms/step - loss: 0.8867 - accuracy: 0.6457 - val_loss: 0.8706 - val_accuracy: 0.6150 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 54s 596ms/step - loss: 0.7983 - accuracy: 0.6625 - val_loss: 1.1985 - val_accuracy: 0.5315 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 57s 624ms/step - loss: 0.7318 - accuracy: 0.6731 - val_loss: 0.8106 - val_accuracy: 0.5896 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 55s 607ms/s

In [9]:
loss, accuracy = denseNet121_model_new.evaluate(test_ds)

26/26 [==============================] - 9s 334ms/step - loss: 0.5523 - accuracy: 0.7464


## MobileNetV2

In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

mobileNetV2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
mobileNetV2.trainable = True
mobileNetV2_model_new = models.Sequential()
mobileNetV2_model_new.add(mobileNetV2)
mobileNetV2_model_new.add(layers.GlobalAveragePooling2D())
mobileNetV2_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
mobileNetV2_model_new.add(layers.Dropout(0.3))
mobileNetV2_model_new.add(layers.Dense(5, activation='softmax'))  

9406464/9406464 [==============================] - 1s 0us/step


In [11]:
mobileNetV2_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [12]:
mobileNetV2_new = mobileNetV2_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 82s 603ms/step - loss: 2.5338 - accuracy: 0.5362 - val_loss: 2.2594 - val_accuracy: 0.4806 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 54s 590ms/step - loss: 1.1947 - accuracy: 0.6334 - val_loss: 2.1568 - val_accuracy: 0.4661 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 54s 591ms/step - loss: 0.8408 - accuracy: 0.6616 - val_loss: 2.6449 - val_accuracy: 0.4818 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 54s 593ms/step - loss: 0.7095 - accuracy: 0.6847 - val_loss: 14.4119 - val_accuracy: 0.1646 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 53s 586ms/step - loss: 0.6777 - accuracy: 0.6926 - val_loss: 1.1453 - val_accuracy: 0.6211 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 53s 584ms/step - loss: 0.6444 - accuracy: 0.7021 - val_loss: 4.9460 - val_accuracy: 0.2494 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 55s 606ms/s

In [14]:
loss, accuracy = mobileNetV2_model_new.evaluate(test_ds)

26/26 [==============================] - 3s 102ms/step - loss: 1.0359 - accuracy: 0.6383
